In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

batch_size = 128
epochs = 10
seed = 1
log_interval = 10
torch.manual_seed(seed)
device = "cpu"

In [2]:
from Tars.distributions import NormalModel, BernoulliModel
from Tars.distributions.divergences import KullbackLeibler
from Tars.models import VAE

In [3]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
x_dim = 784
y_dim = 10
z_dim = 64

# inference model q(z|x,y)
class Inference(NormalModel):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x","y"], var=["z"])

        self.fc1 = nn.Linear(x_dim+y_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, z_dim)

    def forward(self, x, y):
        h1 = F.relu(self.fc1(torch.cat([x, y], 1)))
        return self.fc21(h1), F.softplus(self.fc22(h1))
    
# generative model p(x|z,y)    
class Generator(BernoulliModel):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z","y"], var=["x"])

        self.fc3 = nn.Linear(z_dim+y_dim, 400)
        self.fc4 = nn.Linear(400, x_dim)

    def forward(self, z, y):
        h3 = F.relu(self.fc3(torch.cat([z, y], 1)))
        return F.sigmoid(self.fc4(h3))
    
# prior model p(z|y)
class Prior(NormalModel):
    def __init__(self):
        super(Prior, self).__init__(cond_var=["y"], var=["z"])

        self.fc1 = nn.Linear(y_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, z_dim)

    def forward(self, y):
        h1 = F.relu(self.fc1(y))
        return self.fc21(h1), F.softplus(self.fc22(h1))

In [5]:
p = Generator()
q = Inference()
prior = Prior()

p.to(device)
q.to(device)
prior.to(device)

Prior(
  (fc1): Linear(in_features=10, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=64, bias=True)
  (fc22): Linear(in_features=400, out_features=64, bias=True)
)

In [6]:
kl = KullbackLeibler(q, prior)
model = VAE(q, p, regularizer=kl, optimizer=optim.Adam, optimizer_params={"lr":1e-3})

In [7]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data_x, data_y) in enumerate(train_loader):
        data_x = data_x.to(device)
        data_y = torch.eye(10)[data_y].to(device)
        lower_bound, loss = model.train({"x": data_x.view(-1, 784), "y": data_y})
        train_loss += loss
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data_x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [8]:
def test(epoch):
    test_loss = 0
    for i, (data_x, data_y) in enumerate(test_loader):
        data_x = data_x.to(device)
        data_y = torch.eye(10)[data_y].to(device)
        lower_bound, loss = model.test({"x": data_x.view(-1, 784), "y": data_y})
        test_loss += loss
        z = prior.sample({"y": data_y})
        recon_batch = p.sample_mean(z)
        if i == 0:
            n = min(data_x.size(0), 8)
            comparison = torch.cat([data_x[:n],
                                  recon_batch.view(batch_size, 1, 28, 28)[:n]])
            save_image(comparison.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [9]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        plot_number = 4
        sample_z = 0.3 * torch.randn(64, z_dim).to(device)
        sample_y = torch.eye(10)[[plot_number]*64].to(device)
        
        sample = p.sample_mean({"z":sample_z, "y":sample_y}).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 547.325562
Train Epoch: 1 [1280/60000 (2%)]	Loss: 316.093933
Train Epoch: 1 [2560/60000 (4%)]	Loss: 236.016205
Train Epoch: 1 [3840/60000 (6%)]	Loss: 224.145859
Train Epoch: 1 [5120/60000 (9%)]	Loss: 214.820953
Train Epoch: 1 [6400/60000 (11%)]	Loss: 218.296539
Train Epoch: 1 [7680/60000 (13%)]	Loss: 206.853302
Train Epoch: 1 [8960/60000 (15%)]	Loss: 195.152634
Train Epoch: 1 [10240/60000 (17%)]	Loss: 196.146439
Train Epoch: 1 [11520/60000 (19%)]	Loss: 193.313904
Train Epoch: 1 [12800/60000 (21%)]	Loss: 178.211395
Train Epoch: 1 [14080/60000 (23%)]	Loss: 170.914291
Train Epoch: 1 [15360/60000 (26%)]	Loss: 172.332901
Train Epoch: 1 [16640/60000 (28%)]	Loss: 168.993408
Train Epoch: 1 [17920/60000 (30%)]	Loss: 165.227051
Train Epoch: 1 [19200/60000 (32%)]	Loss: 161.636200
Train Epoch: 1 [20480/60000 (34%)]	Loss: 159.586349
Train Epoch: 1 [21760/60000 (36%)]	Loss: 150.208115
Train Epoch: 1 [23040/60000 (38%)]	Loss: 149.417862
Train Epoch: 1 [24320/60000 

Process Process-3:
Traceback (most recent call last):
  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


Train Epoch: 2 [47360/60000 (79%)]	Loss: 103.692680


  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/masa/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 